In [1]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 3.7 MB/s eta 0:00:00


In [2]:
import os

os.environ["GROQ_API_KEY"] = "api key"


In [4]:
from groq import Groq
import os, requests

# --- Must already be set: os.environ["GROQ_API_KEY"] = "..."
if "GROQ_API_KEY" not in os.environ:
    raise RuntimeError("Set GROQ_API_KEY first")

# 1) Fetch currently available models from Groq
models_url = "https://api.groq.com/openai/v1/models"
headers = {"Authorization": f"Bearer {os.environ['GROQ_API_KEY']}"}
resp = requests.get(models_url, headers=headers, timeout=30)
resp.raise_for_status()
available = [m["id"] for m in resp.json().get("data", [])]

print("✅ Available models (first 15):")
print(available[:15])

# 2) Pick best available model (prefer 70B versatile, fallback to 8B instant)
preferred_order = [
    "llama-3.3-70b-versatile",  # recommended replacement for old 70b-8192
    "llama-3.1-8b-instant",     # lightweight/faster fallback
]

MODEL = next((m for m in preferred_order if m in available), None)
if not MODEL:
    # fallback: just pick the first available model
    MODEL = available[0]

print(f"\n✅ Using model: {MODEL}")

client = Groq()

# ------------------------------------------------
# ROOT PERSONA / SYSTEM PROMPT
# ------------------------------------------------
ROOT_PERSONA = """
You are Arjun Iyer, a senior full-stack and GenAI engineer based in India.

Tone:
- Direct, professional, strict on correctness.

Rules:
1) Think step-by-step privately.
2) Do NOT reveal chain-of-thought.
3) Output ONLY:
   - Reasoning Summary
   - Final Answer
   - Verification Checklist

Patterns:
- Cognitive Verifier Pattern
- Question Refinement Pattern
- Provide New Information and Ask Questions Pattern
- RGC Framework (Role–Goal–Constraints)
"""

def run_prompt(title, user_prompt):
    print(f"\n{'='*80}\n🧪 {title}\n{'='*80}\n")
    completion = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": ROOT_PERSONA.strip()},
            {"role": "user", "content": user_prompt.strip()}
        ],
        temperature=0.2
    )
    print(completion.choices[0].message.content)

# ------------------------------------------------
# 1) ZERO-SHOT
# ------------------------------------------------
run_prompt(
    "ZERO-SHOT PROMPTING",
    """
Design a secure authentication flow for a React + Node.js web application.

Constraints:
- Include JWT lifecycle
- Mention validation and verification
- Include verification checklist
"""
)

# ------------------------------------------------
# 2) QUESTION REFINEMENT
# ------------------------------------------------
run_prompt(
    "QUESTION REFINEMENT PATTERN",
    """
Make login secure for my web app.

Constraints:
- The request is intentionally vague
- Apply Question Refinement Pattern
- Ask at most 2 clarifying questions
- Then proceed with assumptions
"""
)

# ------------------------------------------------
# 3) ONE-SHOT
# ------------------------------------------------
run_prompt(
    "ONE-SHOT PROMPTING",
    """
Example Input:
Secure an API endpoint.

Example Output:
Reasoning Summary:
- Auth, validation, rate limiting.
Final Answer:
- JWT middleware
- Input validation
Verification Checklist:
- 401 on invalid token

NOW DO THIS:
Design a secure password reset flow for a web application.
"""
)

# ------------------------------------------------
# 4) FEW-SHOT
# ------------------------------------------------
run_prompt(
    "FEW-SHOT PROMPTING",
    """
Example 1:
Design caching strategy.
Output:
- Redis cache with TTL

Example 2:
Design DB schema.
Output:
- Normalized tables with indexes

NOW DO THIS:
Design a RAG pipeline for a company FAQ chatbot.
"""
)

# ------------------------------------------------
# 5) RGC FILL-IN-THE-BLANK
# ------------------------------------------------
run_prompt(
    "RGC FILL-IN-THE-BLANK",
    """
Fill the RGC template for this task:
"Build a rate-limited FastAPI endpoint"

RGC TEMPLATE:
[R] ROLE:
[G] GOAL:
[C] CONSTRAINTS:
- Format:
- Must include:
- Must avoid:
- Verification:
"""
)

print("\n✅ Prompt Engineering Activity Completed Successfully")


✅ Available models (first 15):
['llama-3.3-70b-versatile', 'allam-2-7b', 'canopylabs/orpheus-v1-english', 'openai/gpt-oss-20b', 'groq/compound', 'whisper-large-v3-turbo', 'meta-llama/llama-4-maverick-17b-128e-instruct', 'qwen/qwen3-32b', 'canopylabs/orpheus-arabic-saudi', 'groq/compound-mini', 'moonshotai/kimi-k2-instruct-0905', 'llama-3.1-8b-instant', 'openai/gpt-oss-120b', 'whisper-large-v3', 'openai/gpt-oss-safeguard-20b']

✅ Using model: llama-3.3-70b-versatile

🧪 ZERO-SHOT PROMPTING

**Reasoning Summary**
To design a secure authentication flow for a React + Node.js web application, we will implement a JSON Web Token (JWT) based authentication system. The flow will involve user registration, login, and authentication using JWT. We will ensure validation and verification at each step to prevent unauthorized access.

**Final Answer**
The secure authentication flow will consist of the following steps:
1. **User Registration**: The user will register by providing a username, email, and